# Chart Review

In this analysis, we review the medical record for the two patients shown in Figure 4.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
from collections import Counter
import ipywidgets as widgets
import tables
import psycopg2
import numpy as np
import pickle, shelve

In [ ]:
icu_id = 0 # replace with desired ID
# The two IDs we investigated are loaded below:
data_prefix = ''
datapath = '<REPLACE WITH OUTPUT PATH>'
data_dict = shelve.open("{}/{}data_dict.db".format(datapath, data_prefix), flag='r')
best_results = data_dict['best_results']
test_idx_flat = data_dict['test_idx_flat']
icu_id1 = test_idx_flat[1265] # from surprisingly positive outcomes heuristic
test_idx_flat_672_5 = []
for samp_idx in range(test_samps.shape[0]):
    for time_idx in range(test_samps.shape[1]):
        if test_samps[samp_idx,time_idx,2]==672 and test_samps[samp_idx,time_idx,1]==5:
            icustayid = test_idx_flat[samp_idx]
            test_idx_flat_672_5.append(icustayid)
            break
test_idx_flat_672_5 = np.array(test_idx_flat_672_5)
icu_id2 = test_idx_flat_672_5[12] # from surprisingly aggressive treatments heuristic

In [ ]:
sqluser = '<REPLACE WITH USER>'
dbname = 'mimic'
schema_name = 'mimiciii'

con = psycopg2.connect(dbname=dbname, user=sqluser, host='/var/run/postgresql')
cur = con.cursor()

cur.execute('SET search_path to ' + schema_name)

In [ ]:
def query_mimic(query):
    cur.execute('SET search_path to ' + schema_name)
    return pd.read_sql_query(query, con)

In [ ]:
def get_hadm_from_icu(icustay_id):
    icustay_id += 200000
    hadm = query_mimic("select hadm_id from icustays where icustay_id = {}".format(icustay_id))
    return hadm

In [ ]:
this_hadm_id = get_hadm_from_icu(icu_id).values[0][0]
this_subject_id = query_mimic("select subject_id from admissions where hadm_id = {}".format(this_hadm_id)).values[0][0]

First, we check to confirm that this is the only ICU stay for this particular patient. Then, we check the records against the start/end times of our observed trajectory.

In [ ]:
query_mimic("select * from admissions where subject_id = {}".format(this_subject_id))

In [ ]:
fpath = '<REPLACE WITH DATA PATH>'
raw = pd.read_csv("{}/mimic-table.csv".format(fpath)) #MIMICtable

this_patient_id = icu_id

start_chart_time = raw[raw.icustayid == this_patient_id].charttime.iloc[0]
end_chart_time = raw[raw.icustayid == this_patient_id].charttime.iloc[-1]

import datetime
print("Range of Observed / Model-Based Trajectories is given by:")
print(datetime.datetime.fromtimestamp(start_chart_time))
print(datetime.datetime.fromtimestamp(end_chart_time))

In [ ]:
# visualize the notes for a given patient
def visualize_notes(hadm_id):

    # When did this patient arrive (useful for getting first 48 hours)
    admittime = query_mimic("select admittime from admissions where hadm_id="+str(hadm_id)).admittime.values[0]

    query = "select dob, ethnicity, gender, admission_type, admission_location, diagnosis, discharge_location,\
        hospital_expire_flag, marital_status, religion, insurance from patients, admissions\
                              where patients.subject_id=admissions.subject_id and admissions.hadm_id=" + str(hadm_id)
    backgrounds = query_mimic(query)
    #backgrounds['age'] = (backgrounds.dob.values[0] - admittime).total_seconds()/(3600*24*365)
   
    # display the demographics / admission info
    display(backgrounds.T)

    # Get the notes for this patient
    notes_subject = query_mimic("select text, row_id, category, charttime from noteevents where hadm_id="+str(hadm_id)) 
    #notes.loc[notes.hadm_id==hadm_id]

    # How many notes for each category?
    cat_counts = Counter(notes_subject.category.values)
    
    # Build a list of widgets.Accordion objects that we will make into a widgets.Tabs
    accordions = []
    categories = []
    for category,count in sorted(cat_counts.items(), key=lambda t:t[1], reverse=True):    
        # Only notes for this category
        notes_cat = notes_subject.loc[notes_subject.category==category]

        # Sort by time
        notes_cat = notes_cat.sort_values('charttime')

        # Print the data in a structured output
        titles = []
        texts = []
        for num,(i,row) in enumerate(notes_cat.iterrows()):
            # Format the text with additional metadata
            time_offset = (row.charttime - admittime).total_seconds()/3600.
            if np.isnan(time_offset):
                time_offset = 'n/a'
            else:
                time_offset = int(time_offset)
            
            # Only first 48 hours of data
            text = '%s Note #%d, ID %d (%s Hours) (%s chartime)\n\n%s' % (category,
                                                                          num,
                                                                          row.row_id,
                                                                          time_offset,
                                                                          row.charttime,
                                                                          row.text )
            snippet = text[:200]

            # widgets object to display the full text
            collapsable_text = widgets.Output()
            collapsable_text.append_stdout(row.text)

            # Save the formatted data that we will feed in to the widgets.accordian
            titles.append(snippet)
            texts.append(collapsable_text)

        # Display the data beautifully
        accordion = widgets.Accordion(children=texts)
        for i,title in enumerate(titles):
            accordion.set_title(i, title)
        accordion.selected_index = None
    
        # Save this object to be displayed in the widgets.Tab
        accordions.append(accordion)
        categories.append(category)
    
    # This is going to hold all the stuff!
    tab = widgets.Tab()
    tab.children = accordions
    for i,category in enumerate(categories):
        tab.set_title(i, category)
    
    display(tab)
    return tab


In [ ]:
tab = visualize_notes(this_hadm_id)